In [1]:
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score


In [2]:
# Load JSON data
with open('tracker_memory.json', 'r') as f:
    tracker_memory = json.load(f)

with open('user_data.json', 'r') as f:
    user_data = json.load(f)


In [3]:
# Calculate average calorie intake from tracker_memory
def calculate_avg_calories(tracker_memory, user_name):
    total_calories = 0
    count = 0
    for date, records in tracker_memory.items():
        for record in records:
            if record['user_name'] == user_name:
                total_calories += record['calories']
                count += 1
    return total_calories / count if count > 0 else 0

# Calculate TDEE
def calculate_tdee(user, avg_calories):
    weight = float(user['weight'].split()[0])
    height = float(user['tall'].split()[0])
    age = int(user['age'])
    gender = user['gender']
    activity_level = user['ActiveLevel']

    # Map activity levels
    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725
    }

    bmr = (10 * weight + 6.25 * height - 5 * age + (5 if gender == 'male' else -161))
    tdee = bmr * activity_factors.get(activity_level, 1.2)
    return tdee


In [4]:
def preprocess_data(user_data, tracker_memory):
    X = []  # Features
    y = []  # Labels

    for user in user_data:
        user_name = user['user_name']
        avg_calories = calculate_avg_calories(tracker_memory, user_name)
        if avg_calories == 0:
            continue  # Skip users with no tracking data

        tdee = calculate_tdee(user, avg_calories)
        goal_weight = user['goal_weight']
        current_weight = float(user['weight'].split()[0])
        
        # Calculate days to goal
        daily_deficit = avg_calories - tdee
        if daily_deficit == 0:
            continue  # Avoid division by zero
        days_to_goal = abs((current_weight - goal_weight) * 7700 / daily_deficit)

        # Encode activity level
        activity_levels = {"sedentary": 0, "lightly active": 1, "moderately active": 2, "very active": 3}
        activity_level_encoded = activity_levels.get(user['ActiveLevel'], 0)

        # Append data
        X.append([avg_calories, current_weight, goal_weight, activity_level_encoded])
        y.append(days_to_goal)

    return np.array(X), np.array(y)

X, y = preprocess_data(user_data, tracker_memory)


In [5]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for days prediction
])

model.compile(optimizer='adam', loss='mse')


In [7]:


# Train the model
history = model.fit(
    X_train, y_train,
    epochs=3000,
    batch_size=16,
    validation_split=0.2,
    verbose=1,
)


Epoch 1/3000

35/35 [==============================] - 1s 5ms/step - loss: 3319.5820 - val_loss: 2990.9377
Epoch 2/3000
35/35 [==============================] - 0s 2ms/step - loss: 3130.1418 - val_loss: 2737.2954
Epoch 3/3000
35/35 [==============================] - 0s 2ms/step - loss: 2768.0640 - val_loss: 2287.4170
Epoch 4/3000
35/35 [==============================] - 0s 2ms/step - loss: 2233.2798 - val_loss: 1715.8156
Epoch 5/3000
35/35 [==============================] - 0s 2ms/step - loss: 1673.1771 - val_loss: 1278.8239
Epoch 6/3000
35/35 [==============================] - 0s 2ms/step - loss: 1323.1051 - val_loss: 1091.4546
Epoch 7/3000
35/35 [==============================] - 0s 2ms/step - loss: 1172.6616 - val_loss: 1004.8335
Epoch 8/3000
35/35 [==============================] - 0s 2ms/step - loss: 1091.6063 - val_loss: 941.1214
Epoch 9/3000
35/35 [==============================] - 0s 2ms/step - loss: 1011.8497 - val_loss: 885.2937
Epoch 10/3000
35/35 [==========================

In [8]:
# Evaluate model
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss (MSE): {test_loss}")

# Predict
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")


6/6 [==============================] - 0s 2ms/step - loss: 63.9403
Test Loss (MSE): 63.9402961730957
6/6 [==============================] - 0s 1ms/step
Mean Absolute Error (MAE): 5.54
R-squared (R²): 0.93


In [12]:
# Prediksi waktu untuk user tertentu
sample_user = user_data[8]  # Ambil data user pertama dalam user_data
avg_calories = calculate_avg_calories(tracker_memory, sample_user['user_name'])
tdee = calculate_tdee(sample_user, avg_calories)

# Encode tingkat aktivitas
activity_levels = {"sedentary": 0, "lightly active": 1, "moderately active": 2, "very active": 3}
activity_level_encoded = activity_levels.get(sample_user['ActiveLevel'], 0)

# Buat input untuk prediksi
sample_input = np.array([[avg_calories, float(sample_user['weight'].split()[0]), 
                          sample_user['goal_weight'], activity_level_encoded]])
sample_input_scaled = scaler.transform(sample_input)

# Prediksi menggunakan model yang sudah dilatih
predicted_days = model.predict(sample_input_scaled)
print(f"Estimated days to reach goal weight for {sample_user['user_name']}: {predicted_days[0][0]:.2f} days")


1/1 [==============================] - 0s 21ms/step
Estimated days to reach goal weight for User9: 11.84 days


In [10]:
# Save the model
#model.save('my_model')  # Save in SavedModel format
#model.save('my_model.h5')  # Save in HDF5 format
